## Importamos las librerias necesarias

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.cluster import KMeans

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler

from sklearn.model_selection import cross_val_score, train_test_split
from xgboost import XGBClassifier

import numpy as np
from xgboost import XGBClassifier
from bayes_opt import BayesianOptimization
from sklearn.model_selection import cross_val_score
import xgboost as xgb
import pandas_profiling

## Leemos y procesamos las datos para unir la informacion estatica con la info transaccional

In [ ]:
df = pd.read_csv('data/DT19_Datos_Var_Rpta_train.csv')
df_detail = pd.read_csv('data/DT19_Datos_transaccionales_train.csv')
df_id = pd.read_csv('data/DT19_IDs_predict.csv')
df_trans_predict = pd.read_csv('data/DT19_Datos_transaccionales_predict.csv')
df['train'] = 1
df_id['train'] = 0
print(df_detail.shape, df_trans_predict.shape)
print(df_id.shape, df.shape)
df = pd.concat([df, df_id])
df_detail = pd.concat([df_detail, df_trans_predict])
df_meta = pd.read_csv('data/DT19_maestro_cdgtrn_cdgrpta.csv')
df_meta.fillna(0, inplace=True)
df_detail['cdgrpta'] = df_detail['cdgrpta'].astype(float)
df_detail['cdgtrn'] = df_detail['cdgtrn'].astype(float)
df_detail['cdgrpta'] = df_detail['cdgrpta'].astype(str)
df_detail['cdgtrn'] = df_detail['cdgtrn'].astype(str)
print (df_detail.shape)
df_detail = pd.merge(df, df_detail, on=['id'])
print (df_detail.shape)
df_detail = pd.merge(df_detail, df_meta, on=['canal','disposit','cdgtrn','cdgrpta'])
df_detail['fecha_trxn'] =  pd.to_datetime(df_detail['fecha_trxn'])
df_detail['year'] = df_detail['fecha_trxn'].dt.year
df_detail['month'] = df_detail['fecha_trxn'].dt.month
df_detail['day'] = df_detail['fecha_trxn'].dt.day
df_detail['quarter'] = df_detail['fecha_trxn'].dt.quarter
df_detail['dayofweek'] = df_detail['fecha_trxn'].dt.dayofweek
df_detail['hour'] = df_detail['fecha_trxn'].dt.hour
df_detail['second'] = df_detail['fecha_trxn'].dt.second
df_detail['weekofyear'] = df_detail['fecha_trxn'].dt.strftime("%V")
df_detail.head()

## Guardamos los dataframe finales para no tener que repetir este proceso en futuras cargas

In [ ]:
pd.to_pickle('df_detail.plk')
pd.to_pickle('df_final.plk')
pd.to_pickle('df.plk')

## Leemos los dataframes preprocesados  

In [ ]:
df_detail = pd.read_pickle('df_detail.plk')
df_final = pd.read_pickle('df_final.plk')
df = pd.read_pickle('df.plk')
print ( df.shape, df_final.shape)
df_final.head()

### EDA inicial

In [ ]:
df_eda = df[df['train'] == 1]
df_eda['f_analisis'] = df_eda['f_analisis'].astype(str)
df_eda['year'] = [ value[:4] for value in df_eda['f_analisis']]
df_eda['year'] = df_eda['year'].astype(int)
df_eda['month'] = [ value[4:] for value in df_eda['f_analisis'] ]
df_eda['month'] = df_eda['month'].astype(int)
df_eda.fillna(0, inplace=True)
profile = df_eda.profile_report(title='BancolombiaEDA')
profile.to_file(output_file="bancolombiaEDA.html")

## Creamos las caracteristicas (feature engineer)

### Agrupamiento de sesiones unicas por cada una de las variables categoricas

In [ ]:
columns = ['canal', 'disposit','grupo_descrp_trxn',
           'descrip_trxn','clasif_trxn','clasif_cod_rpta','producto_asociado',
          'descripcion_grupo', 'descrip_cod_rpta','grupo_modifcado','culpa_banco']

df_final = df.copy()
##Cantidad de sesiones unicas por canal, dispositivo
for column in columns:
    print (column)
    df_final_group = df_detail.groupby(['id',column], as_index=False).agg({'sesion':'nunique'})
    for value in df_final_group[column].unique():
        data = df_final_group[df_final_group[column] == value]
        data.drop([column], axis=1, inplace=True)
        df_final = pd.merge(df_final, data, on=['id'], how='left')
        df_final.fillna(0, inplace=True)
        df_final.rename(columns={'sesion':str(column) + '_' + str(value) + '_cantidad_sesiones' }, inplace=True)

        
print(df_final.shape)

### Calculo de factores estadísticos sum, mean, max, min y std

In [ ]:
## Valor sum, max, min,mean y std del valor de transaccion
sum_vlr = df_detail.groupby(['id'], as_index=False).sum()[['id','vlrtran', 'sesion', 'year', 'month',
       'day', 'hour', 'minute', 'second', 'quarter', 'dayofweek']]
df_final = pd.merge(df_final, sum_vlr, on=['id'], how='left')
df_final.rename(columns={'vlrtran':'vlrtran_sum',
                        'sesion':'sesion_sum',
                        'year':'year_sum',
                        'month':'month_sum',
                        'day':'day_sum',
                        'hour':'hour_sum',
                        'minute':'minute_sum',
                        'second':'second_sum',
                        'quarter':'quarter_sum',
                        'dayofweek':'dayofweek_sum'}, inplace=True)
print(df_final.shape)
min_vlr = df_detail.groupby(['id'], as_index=False).min()[['id','vlrtran', 'sesion', 'year', 'month',
       'day', 'hour', 'minute', 'second', 'quarter', 'dayofweek']]
df_final = pd.merge(df_final, min_vlr, on=['id'], how='left')
df_final.rename(columns={'vlrtran':'vlrtran_min',
                        'sesion':'sesion_min',
                        'year':'year_min',
                        'month':'month_min',
                        'day':'day_min',
                        'hour':'hour_min',
                        'minute':'minute_min',
                        'second':'second_min',
                        'quarter':'quarter_min',
                        'dayofweek':'dayofweek_min'}, inplace=True)
print(df_final.shape)

max_vlr = df_detail.groupby(['id'], as_index=False).max()[['id','vlrtran', 'sesion', 'year', 'month',
       'day', 'hour', 'minute', 'second', 'quarter', 'dayofweek']]
df_final = pd.merge(df_final, max_vlr, on=['id'], how='left')
df_final.rename(columns={'vlrtran':'vlrtran_max',
                        'sesion':'sesion_max',
                        'year':'year_max',
                        'month':'month_max',
                        'day':'day_max',
                        'hour':'hour_max',
                        'minute':'minute_max',
                        'second':'second_max',
                        'quarter':'quarter_max',
                        'dayofweek':'dayofweek_max'}, inplace=True)
print(df_final.shape)


mean_vlr = df_detail.groupby(['id'], as_index=False).mean()[['id','vlrtran', 'sesion', 'year', 'month',
       'day', 'hour', 'minute', 'second', 'quarter', 'dayofweek']]
df_final = pd.merge(df_final, mean_vlr, on=['id'], how='left')
df_final.rename(columns={'vlrtran':'vlrtran_mean',
                        'sesion':'sesion_mean',
                        'year':'year_mean',
                        'month':'month_mean',
                        'day':'day_mean',
                        'hour':'hour_mean',
                        'minute':'minute_mean',
                        'second':'second_mean',
                        'quarter':'quarter_mean',
                        'dayofweek':'dayofweek_mean'}, inplace=True)
print(df_final.shape)

std_vlr = df_detail.groupby(['id'], as_index=False).std()[['id','vlrtran', 'sesion', 'year', 'month',
       'day', 'hour', 'minute', 'second', 'quarter', 'dayofweek']]
df_final = pd.merge(df_final, std_vlr, on=['id'], how='left')
df_final.rename(columns={'vlrtran':'vlrtran_std',
                        'sesion':'sesion_std',
                        'year':'year_std',
                        'month':'month_std',
                        'day':'day_std',
                        'hour':'hour_std',
                        'minute':'minute_std',
                        'second':'second_std',
                        'quarter':'quarter_std',
                        'dayofweek':'dayofweek_std'}, inplace=True)
print(df_final.shape)

## One hot encoding

In [ ]:
##One-hot encoding de los segmentos
df_ = df.copy()
df_['segmento'] = np.where(df_['segmento'] == 0, 6, df_['segmento'])
dummy = pd.get_dummies(df_['segmento'], prefix='segmento',  drop_first=True)
df_ = pd.concat([df[['id']], dummy], axis=1)
df_final = pd.merge(df_final, df_, on=['id'], how='left')

print(df_final.shape)

### Calculado diferencia de fechas

In [ ]:
df_final['f_analisis'] = df_final['f_analisis'].astype(str)
df_final['year'] = [ value[:4] for value in df_final['f_analisis']]
df_final['year'] = df_final['year'].astype(int)
df_final['month'] = [ value[4:] for value in df_final['f_analisis'] ]
df_final['month'] = df_final['month'].astype(int)
df_final.fillna(0, inplace=True)

##Diff fechas
df_final['max_diff_year'] = df_final['year'] - df_final['year_max']
df_final['min_diff_year'] = df_final['year'] - df_final['year_min']
df_final['mean_diff_year'] = df_final['year'] - df_final['year_mean']
df_final['max_diff_month'] = df_final['month'] - df_final['month_max']
df_final['min_diff_month'] = df_final['month'] - df_final['month_min']

### Porcentage de cada una de las posibles salidas

In [ ]:
fail_detail = df_detail["var_rpta"].value_counts(normalize=True)[1]
no_fail_detail = df_detail["var_rpta"].value_counts(normalize=True)[0]
df_final['fail_detail'] = np.where(df_final['var_rpta'] == 1, fail_detail, fail_detail)
df_final['no_fail_detail'] = np.where(df_final['var_rpta'] == 0, no_fail_detail, no_fail_detail)
fail_detail = df["var_rpta"].value_counts(normalize=True)[1]
no_fail_detail = df["var_rpta"].value_counts(normalize=True)[0]
df_final['percentage_fail'] = np.where(df_final['var_rpta'] == 1, fail_detail, fail_detail)
df_final['percentage_no_fail'] = np.where(df_final['var_rpta'] == 0, no_fail_detail, no_fail_detail)

### Importancia de las variables para clusterizar

In [ ]:
model = XGBClassifier(
    learning_rate =0.1,
    n_estimators=100,
    max_depth=5,
    min_child_weight=1,
    gamma=0,
    subsample=0.8,
    colsample_bytree=0.8,
    objective= 'binary:logistic',
    nthread=4,
    scale_pos_weight=2,
    seed=27)
model.fit(X_sc,y)
score_train = model.score(X_train,y_train)
score_test = model.score(X_test,y_test)
print("Score train is: {} and Score test is {}".format(score_train, score_test))

In [ ]:
#Mirando las variables explicativas
predictors = X.columns
features = list(zip(predictors,model.feature_importances_))
variables = pd.DataFrame(np.array(features).reshape(X.shape[1],2), columns = ["Variable","Peso"])
variables['Peso'] = variables['Peso'].astype(float)
variables.sort_values(["Peso"], ascending=False, inplace=True)
variables.head()

### Segmentacion propia

In [ ]:
##Cluster con k-means
variable = list(variables.head(80)['Variable'].values)
variable.append('id')
variable.append('var_rpta')
df_final_t = df_final[variable].groupby('id').sum()
X = df_final_t

X.dropna(inplace=True)
sc = MinMaxScaler()
X_sc = sc.fit_transform(X)

##Agrupamiento por KMeans con 5 clusters
km = KMeans(n_clusters=250, random_state=5)
km.fit(X_sc)
labels = km.labels_
mm = pd.DataFrame(km.fit_predict(X), columns=['labels'])
silhouette = (X, mm['labels'])

# Agregar cluster a los clientes
df_final_cluster = df_final.copy()
df_final["Medium Cluster"] = km.predict(X)

### EDA Final

In [ ]:
variables_ = list(variables[variables['Peso'] > 0.005]['Variable'].values)
df_final = df_final[variables_]
df_final = pd.concat([df[['id','var_rpta','train']].reset_index(drop=True), df_final], axis=1)
print(df_final.shape)
df_final.head()

# Evaluacion

In [ ]:
df_final_ = df_final[df_final['train'] == 1]

In [ ]:
X = df_final_.drop(["var_rpta",'id','f_analisis','train'],axis=1)
y = df_final_['var_rpta']

In [ ]:
X.head()

In [ ]:
#Estandarizando los datos
sc = StandardScaler()
X_sc = sc.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X_sc, y,shuffle=False,test_size=0.2)
models = [LogisticRegression(), RandomForestClassifier(), XGBClassifier(), AdaBoostClassifier()]

## Evaluacion de los modelos posibles

In [ ]:
cv_score = []
for model in models:
    model = model
    score_model = cross_val_score(model,X_sc,y, cv=2,scoring='roc_auc')
    score_final = score_model.mean()
    cv_score.append(score_final)
cv_score

## Modelo

In [ ]:
cv_score = []
model_e = XGBClassifier()
score_model = cross_val_score(model_e,X_sc,y, cv=3,scoring='roc_auc')
score_final = score_model.mean()
cv_score.append(score_final)
print(str(cv_score))

## Hypertuning

In [ ]:
pbounds = {
    'learning_rate': (0.01, 1.0),
    'n_estimators': (100, 1000),
    'max_depth': (3,10),
    'subsample': (0.5, 1.0),  # Change for big datasets
    'colsample': (0.5, 1.0),  # Change for datasets with lots of features
    'gamma': (0, 5)}
 
def xgboost_hyper_param(learning_rate,
                        n_estimators,
                        max_depth,
                        subsample,
                        colsample,
                        gamma):
 
    max_depth = int(max_depth)
    n_estimators = int(n_estimators)
 
    clf = XGBClassifier(
        max_depth=max_depth,
        learning_rate=learning_rate,
        n_estimators=n_estimators,
        gamma=gamma)
    return np.mean(cross_val_score(clf, X_train, y_train, cv=3, scoring='roc_auc'))
 
optimizer = BayesianOptimization(
    f=xgboost_hyper_param,
    pbounds=pbounds,
    random_state=1,
)
optimizer.maximize()

## Prediccion de los nuevos valores

In [ ]:
df_final_p = df_final[df_final['train'] == 0]
X_p = df_final_p.drop(["var_rpta",'f_analisis','id','train'],axis=1)
y_p = df_final_p['var_rpta']

In [ ]:
cv_score = []

model_e = XGBClassifier(
    learning_rate =0.1,
    n_estimators=100,
    max_depth=5,
    min_child_weight=1,
    gamma=0,
    subsample=1,
    colsample_bytree=0.8,
    objective= 'binary:logistic',
    nthread=4,
    scale_pos_weight=2,
    seed=27)
score_model = cross_val_score(model_e,X_sc,y, cv=3,scoring='roc_auc')
score_final = score_model.mean()
cv_score.append(score_final)
print(str(cv_score))
model_e.fit(X_sc, y, eval_metric=['auc'])

In [ ]:
#Estandarizando los datos
sc = StandardScaler()
X_sc_p = sc.fit_transform(X_p)
y_pred = model_e.predict_proba(X_sc_p)
y_pred = pd.DataFrame(y_pred, columns=['Pago','probabilidad'])
df_id = pd.read_csv('data/DT19_IDs_predict.csv')
y_pred = pd.concat([df_id[['id']], y_pred[['probabilidad']]], axis=1)
y_pred

In [ ]:
y_pred.to_csv('try1.csv', index=False)